In [ ]:
#| hide
import os
import joblib
import warnings
from execnb.nbio import read_nb, write_nb, new_nb, mk_cell
from nbdev.process import NBProcessor, Processor

# Syncing through nbdev

> Showcasing the syncing capability

In [ ]:
%load_ext nbmodular.core.cell2func

## FunctionNBProcessor

In [ ]:
a = [4, 2, 5]

In [ ]:
a[1] = 5

In [ ]:
a

[4, 5, 5]

In [ ]:
import pdb
class FunctionNBProcessor(Processor):
    def __init__ (self, cell_processor, debug=False):
        self.cell_processor = cell_processor
        self.function_idx = 0
        self.cell_list = []
        self.function_name_to_list_idx = {}
        self.debug=debug
        #for function in self.cell_processor.cell_nodes:
        #    function.cell_idx = None
    
    def cell(self, cell):
        separator = '\n**********************************************************************\n'
        cell_lines = cell.source.splitlines()
        if len(cell_lines) > 0 and '%%function' in cell_lines[0]:
            if self.debug:
                pdb.set_trace()
            if self.function_idx < len(self.cell_processor.cell_nodes):
                this_function = self.cell_processor.cell_nodes[self.function_idx]
                this_function.cell_idx = cell.idx_
                if this_function.name in self.function_name_to_list_idx:
                    list_idx = self.function_name_to_list_idx[this_function.name]
                    self.cell_list[list_idx] = '#| export\n' + this_function.code
                else:
                    list_idx = len(self.cell_list)-1
                    self.cell_list.append (separator + '#| export\n' + this_function.code)
                    self.function_name_to_list_idx[this_function.name] = list_idx
                self.function_idx += 1
        else:
            self.cell_list.append (cell.source)
    def end(self): 
        for cell in self.cell_list:
            print (cell)

## process_notebook

In [ ]:
def process_notebook (cell_processor, debug=False):
    function_processor = FunctionNBProcessor (cell_processor, debug=debug)
    NBProcessor ('./nbdev_sync.ipynb', function_processor).process()

## simple case: each function is written in a single cell

This is markdown

In [ ]:
%%function first_function
a = 3 
print ('a', a)

a 3
Stored the following local variables in the first_function current_values dictionary: ['a']


In [ ]:
# normal code cell
print ('outside', a)

outside 3


In [ ]:
%%function second_function
b = 4
c = a+b
print (a, b, c)

3 4 7
Stored the following local variables in the second_function current_values dictionary: ['b', 'c']


In [ ]:
cell_processor = %cell_processor

In [ ]:
[function['name'] for function in cell_processor.cell_nodes]

['first_function', 'second_function']

In [ ]:
print ('last cell:', a, b, c)

last cell: 3 4 7


In [ ]:
process_notebook (cell_processor)

import os
import joblib
import warnings
from execnb.nbio import read_nb, write_nb, new_nb, mk_cell
from nbdev.process import NBProcessor, Processor
# Syncing through nbdev

> Showcasing the syncing capability
%load_ext nbmodular.core.cell2func
## FunctionNBProcessor
a = [4, 2, 5]
a[1] = 5
a
import pdb
class FunctionNBProcessor(Processor):
    def __init__ (self, cell_processor, debug=False):
        self.cell_processor = cell_processor
        self.function_idx = 0
        self.cell_list = []
        self.function_name_to_list_idx = {}
        self.debug=debug
        #for function in self.cell_processor.cell_nodes:
        #    function.cell_idx = None
    
    def cell(self, cell):
        if self.debug:
            pdb.set_trace()
        separator = '\n**********************************************************************\n'
        cell_lines = cell.source.splitlines()
        if len(cell_lines) > 0 and '%%function' in cell_lines[0]:
            if self.function_idx < len(self.cel

## second case: functions are split into different cells

In [ ]:
%%function myf
print ('hello')
a = 3

hello
Stored the following local variables in the myf current_values dictionary: ['a']


In [ ]:
%print myf

def myf():
    print ('hello')
    a = 3
    return a



In [ ]:
myf_info.original_cell

"print ('hello')\na = 3\n"

In [ ]:
%%function myf --merge
b = 4

Stored the following local variables in the myf current_values dictionary: ['b']
hello
Stored the following local variables in the myf current_values dictionary: ['a', 'b']


In [ ]:
%print myf

def myf():
    print ('hello')
    a = 3
    b = 4
    return b,a



In [ ]:
%%function myf --merge
c = 5
d = a+b+c

Stored the following local variables in the myf current_values dictionary: ['c', 'd']
hello
Stored the following local variables in the myf current_values dictionary: ['a', 'b', 'c', 'd']


In [ ]:
%print myf

def myf():
    print ('hello')
    a = 3
    b = 4
    c = 5
    d = a+b+c
    return d,c,b,a



In [ ]:
myf_info.original_cell

"print ('hello')\na = 3\nb = 4\nc = 5\nd = a+b+c\n"

In [ ]:
process_notebook (cell_processor)

import os
import joblib
import warnings
from execnb.nbio import read_nb, write_nb, new_nb, mk_cell
from nbdev.process import NBProcessor, Processor
# Syncing through nbdev

> Showcasing the syncing capability
%load_ext nbmodular.core.cell2func
## FunctionNBProcessor
a = [4, 2, 5]
a[1] = 5
a
import pdb
class FunctionNBProcessor(Processor):
    def __init__ (self, cell_processor, debug=False):
        self.cell_processor = cell_processor
        self.function_idx = 0
        self.cell_list = []
        self.function_name_to_list_idx = {}
        self.debug=debug
        #for function in self.cell_processor.cell_nodes:
        #    function.cell_idx = None
    
    def cell(self, cell):
        if self.debug:
            pdb.set_trace()
        separator = '\n**********************************************************************\n'
        cell_lines = cell.source.splitlines()
        if len(cell_lines) > 0 and '%%function' in cell_lines[0]:
            if self.function_idx < len(self.cel

### (TODO) Use only the last cell associated with this function, e.g., by replacing previous cells.

## third case: have defined functions in cells

In [ ]:
x=4

In [ ]:
%%function 
def my_defined_function (x, a=3):
    print (x)
    print (a)

4
3
Stored the following local variables in the my_defined_function current_values dictionary: []


In [ ]:
process_notebook (cell_processor, debug=True)

> /tmp/ipykernel_10913/1001492472.py(15)cell()
     13         if self.debug:
     14             pdb.set_trace()
---> 15         separator = '\n**********************************************************************\n'
     16         cell_lines = cell.source.splitlines()
     17         if len(cell_lines) > 0 and '%%function' in cell_lines[0]:



ipdb>  ll


     12     def cell(self, cell):
     13         if self.debug:
     14             pdb.set_trace()
---> 15         separator = '\n**********************************************************************\n'
     16         cell_lines = cell.source.splitlines()
     17         if len(cell_lines) > 0 and '%%function' in cell_lines[0]:
     18             if self.function_idx < len(self.cell_processor.cell_nodes):
     19                 this_function = self.cell_processor.cell_nodes[self.function_idx]
     20                 this_function.cell_idx = cell.idx_
     21                 if this_function.name in self.function_name_to_list_idx:
     22                     list_idx = self.function_name_to_list_idx[this_function.name]
     23                     self.cell_list[list_idx] = '#| export\n' + this_function.code
     24                 else:
     25                     list_idx = len(self.cell_list)-1
     26                     self.cell_list.append (separator + '#| export\n' + this_f

ipdb>  n


> /tmp/ipykernel_10913/1001492472.py(16)cell()
     14             pdb.set_trace()
     15         separator = '\n**********************************************************************\n'
---> 16         cell_lines = cell.source.splitlines()
     17         if len(cell_lines) > 0 and '%%function' in cell_lines[0]:
     18             if self.function_idx < len(self.cell_processor.cell_nodes):



...

ipdb>  


--Return--
None
> /tmp/ipykernel_10913/1001492472.py(30)cell()
     28                 self.function_idx += 1
     29         else:
---> 30             self.cell_list.append (cell.source)
     31     def end(self):
     32         for cell in self.cell_list:



ipdb>  c


> /tmp/ipykernel_10913/1001492472.py(15)cell()
     13         if self.debug:
     14             pdb.set_trace()
---> 15         separator = '\n**********************************************************************\n'
     16         cell_lines = cell.source.splitlines()
     17         if len(cell_lines) > 0 and '%%function' in cell_lines[0]:



ipdb>  q


### (TODO) Debug why the code of last function doesn't appear at all

## end